In [1]:
# python3.6

import random
import sys
from IPython.display import clear_output, display
from paho.mqtt import client as mqtt_client
import os
import time
from datetime import datetime
from jetbot import Robot

robot = Robot()

broker = '127.0.0.1'
port = 1884
topic = "python/mqtt"
# generate client ID with pub prefix randomly
client_id = f'python-mqtt-{random.randint(0, 100)}'

STATE = 0
LAST_STATE = -1
last_k = None
last_direction = None
vel = 0.44
ids_detected = []

def connect_mqtt() -> mqtt_client:
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)

    client = mqtt_client.Client(client_id)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client


def subscribe(client: mqtt_client):
    def on_message(client, userdata, msg):
        global STATE, LAST_STATE, last_k, last_direction, vel,ids_detected
    
        received = msg.payload.decode()
        clear_output(wait=True)
        print("MQTT Broker messages:")
        try:
            #x, y, k = 0
            x = 0.0
            y = 0.0
            k = last_k if last_k is not None else 0.0
            sign = -1
            sign_id = -1
            mqtt_input = received.split(" ")
            instruction = mqtt_input[0]
            params = mqtt_input[1]

            if instruction == "Vision":
                commands = params.split(",")
                x = float(commands[0])
                y = float(commands[1])
                k = float(commands[2])
#                 if last_direction is not None:
#                     if x > 800.0:
#                         if last_direction != "LEFT":
#                             k = new_k
#                     elif x < 400.0:
#                         if last_direction != "RIGHT":
#                             k = new_k
#                 else:
#                     k = new_k
#                 if x > 800.0:
#                     last_direction = "RIGHT"
#                 elif x < 400.0:
#                     last_direction = "LEFT"
#                 else:
#                     last_direction = "CENTER"
                last_k = k
                    
                

            elif instruction == "DeepStream":
                params = params.split(",")
                sign = int(params[0])
                sign_id = int(params[1]) 
            
            print(f"x: {x}, y: {y}, k: {k}")
            print(f"Id's detected: {ids_detected}")
            
            if sign >= 0 and sign_id not in ids_detected:
                ids_detected.append(sign_id)
                print(sign_id)
                STATE = sign+1
                client.disconnect()
            else:
                #LAST_STATE = 0
                robot.left_motor.value = max(min(vel + k, 1.0), 0.0)
                robot.right_motor.value = max(min((vel-0.05) - k, 1.0), 0.0)

#                 print(f"Traffic signs detected = {signs_detected} \t Traffic signs obeyed = {signs_obeyed}")

        except Exception as e:
            print(f"Error trying to find commands")
    client.subscribe(topic)
    client.on_message = on_message


def run():
    global STATE, LAST_STATE, vel
    while STATE == 0:
        client = connect_mqtt()
        subscribe(client)
        client.loop_forever()
        if STATE == 1:
            LAST_STATE = STATE
            print("Parando")
            last = datetime.now()
            robot.left_motor.value = 0.0
            robot.right_motor.value = 0.0
            while (datetime.now() - last).total_seconds() < 3.0:
                continue
            print("Voltando")
            STATE = 0
        elif STATE == 2:
            LAST_STATE = STATE
            print("40km/h")
            vel = 0.34
            STATE = 0
        
        elif STATE == 3:
            LAST_STATE = STATE
            print("60km/h")
            vel = 0.6
            STATE = 0
            
    #         LAST_STATE = STATE

            
            
    print(STATE)

if __name__ == '__main__':
    run()

MQTT Broker messages:
x: 1022.0, y: 237.0, k: 0.04058966715949344
Id's detected: [1, 3, 4, 2, 5, 8, 9, 11, 12, 13, 14]


KeyboardInterrupt: 

In [5]:
from jetbot import Robot

robot = Robot()

In [2]:
robot.stop()

In [3]:
from datetime import datetime
from IPython.display import clear_output, display
last = datetime.now()
while (datetime.now() - last).total_seconds() < 2.0:
    clear_output(wait=True)
#    time.sleep(1)
    display(datetime.now())

datetime.datetime(2022, 10, 31, 22, 46, 57, 217971)